In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from pulp import *

# Initialize problem
problem = LpProblem("University Time Table Scheduling", LpMinimize)

# Decision variables
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
times = ['8:00', '9:00', '10:00', '11:00', '12:00', '1:00', '2:00', '3:00', '4:00']
rooms = ['R1', 'R2', 'R3']
courses = ['C1', 'C2', 'C3', 'C4']
faculties = ['F1', 'F2', 'F3']
num_students = {'C1': 25, 'C2': 30, 'C3': 20, 'C4': 35}
course_fixed_time = {'C1': ('Monday', '8:00'), 'C2': ('Monday', '9:00')}
faculty_pref_time = {('C1', 'F1'): ('Monday', '8:00'), ('C2', 'F2'): ('Monday', '9:00')}
x = LpVariable.dicts("x", [(i, j, k, t) for i in courses for j in rooms for k in times for t in days], cat='Binary')
y = LpVariable.dicts("y", [(i, j, k) for i in courses for j in faculties for k in times], cat='Binary')

# Objective function
problem += lpSum([x[(i, j, k, t)] for i in courses for j in rooms for k in times for t in days])

# Constraints
for j in rooms:
    for k in times:
        for t in days:
            problem += lpSum([x[(i, j, k, t)] * num_students[i] for i in courses]) <= 50

for i in courses:
    problem += lpSum([x[(i, j, k, t)] for j in rooms for k in times for t in days]) == 1

for k in times:
    for t in days:
        for i in courses:
            problem += lpSum([x[(i, j, k, t)] for j in rooms]) <= 1

for j in faculties:
    for k in times:
        problem += lpSum([y[(i, j, k)] for i in courses]) <= 1

for j in faculties:
    for k in times:
        for t in days:
            problem += lpSum([x[(i, j, k, t)] * y[(i, j, k)] for i in courses]) <= 1
print("Faculty preference constraint for", i, j, k, ":", y[(i, j, k)].name, "=", y[(i, j, k)].varValue)


for i in courses:
    for j in faculties:
        for k in times:
            if (i, j) in faculty_pref_time and faculty_pref_time[(i, j)] != (t, k):
                problem += y[(i, j, k)] == 0

for i in courses:
    if i in course_fixed_time:
        t, k = course_fixed_time[i]
        for j in rooms:
            problem += x[(i, j, k, t)] == 1

# Solve problem
problem.solve()

# Print status and value of objective function
print(f"Status: {LpStatus[problem.status]}, Objective Function Value: {problem.objective.value()}")

# Print values of decision variables
for v in problem.variables():
    print(v.name, "=", v.varValue)

KeyError: ('C1', 'F1', '8:00', 'Monday')